# Analise de Vôos

Este notebook tem o objetivo de fazer uma análise exploratoria inicial do arquivo flights-larger.csv a fim de executar algumas transformações para responder algumas perguntas de negócio e desenvolver algumas linhas de análise.

O Spark é uma das tecnologias mais populares e fáceis para processar grandes volumes de dados. Além disso, é fácil para os iniciantes trabalharem com o Spark em comparação com outras tecnologias de computação distribuída. Então, usando a linguagem Pyspark, uma interface Python para Spark, interagiremos com o Spark.

## Primeiro passo
Primeiramente, no prompt de comando da minha máquina Windows host do projeto, digito o comando ***pyspark*** para que o mesmo já abra o jupyter e crie uma Spark Session automaticamente para meu projeto.